In [1]:
#omgamganapathayenamaha
#omgamganapathayenamaha
#omgamganapathayenamaha
#omgamganapathayenamaha
#omgamganapathayenamaha

In [2]:
# Upload training and test data to S3
import boto3
train_bucket = 'ganesa-ml-sagemaker'
train_object = 'binary_text_classification/train/train.csv'
with open('train.csv', 'rb') as fh:
    boto3.Session(region_name='us-east-1').resource('s3').Bucket(train_bucket).Object(train_object).upload_fileobj(fh)

test_bucket = 'ganesa-ml-sagemaker'
test_object = 'binary_text_classification/test/test.csv'
with open('test.csv', 'rb') as fh:
    boto3.Session(region_name='us-east-1').resource('s3').Bucket(test_bucket).Object(test_object).upload_fileobj(fh)

In [3]:
pipeln_bucket = 'ganesa-ml-sagemaker'
pipeln_object = 'binary_text_classification/data_pipeline/data_pipeline'
with open('data_pipeline', 'rb') as fh:
    boto3.Session(region_name='us-east-1').resource('s3').Bucket(pipeln_bucket).Object(pipeln_object).upload_fileobj(fh)

In [22]:
import sagemaker

In [23]:
from sagemaker import get_execution_role
role = get_execution_role()

In [24]:
sess = sagemaker.Session()

In [25]:
sess.boto_region_name

'us-east-1'

In [26]:
container = sagemaker.image_uris.retrieve(framework='xgboost', region=sess.boto_region_name, version='1.2-2')
container

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-2'

In [27]:
output_path='s3://ganesa-ml-sagemaker/binary_text_classification/model/'
base_job_name = 'xgboost-aws'
hyperparameters = {
    'objective': 'binary:logistic',
    'num_round': 100,
    'max_depth': 5,
    'early_stopping_rounds': 10
}
estimator = sagemaker.estimator.Estimator(image_uri=container,
                                         hyperparameters=hyperparameters,
                                         instance_count=1,
                                         instance_type='ml.m5.large',
                                         role=role,
                                         sagemaker_session=sess,
                                         input_mode='File',
                                         output_path=output_path,
                                          base_job_name=base_job_name
                                         )

In [28]:
# content type can be libsvm or csv for XGBoost
s3_training_file_location = 's3://ganesa-ml-sagemaker/binary_text_classification/train/'
s3_validation_file_location = 's3://ganesa-ml-sagemaker/binary_text_classification/test/'
training_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_training_file_location,
    content_type='csv',
    s3_data_type='S3Prefix')

validation_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_validation_file_location,
    content_type='csv',
    s3_data_type='S3Prefix'
)

data_channels = {'train': training_input_config, 'validation': validation_input_config}

In [29]:
estimator.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-aws-2024-06-19-09-25-18-091


2024-06-19 09:25:18 Starting - Starting the training job...
2024-06-19 09:25:34 Starting - Preparing the instances for training...
2024-06-19 09:26:05 Downloading - Downloading input data......
2024-06-19 09:26:50 Downloading - Downloading the training image...
2024-06-19 09:27:41 Training - Training image download completed. Training in progress..[2024-06-19 09:27:43.933 ip-10-0-110-22.ec2.internal:8 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-06-19:09:27:43:INFO] Imported framework sagemaker_xgboost_container.training
[2024-06-19:09:27:43:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-06-19:09:27:43:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-19:09:27:43:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-06-19:09:27:43:INFO] Determined delimiter of CSV input is ','
[2024-06-19:09:27:44:INFO] Determined delimiter of CSV input is ','
[2024-06-19:09:27:44:INFO] Determined delimiter of 

In [30]:
#data_pipeline = None
#pipeln_bucket = 'ganesa-ml-sagemaker'
#pipeln_object = 'binary_text_classification/data_pipeline/data_pipeline'
with open('data_pipeline_downloaded', 'wb') as fw:
    boto3.Session(region_name='us-east-1').resource('s3').Bucket(pipeln_bucket).Object(pipeln_object).download_fileobj(fw)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [31]:
import pickle
class sparse_to_dense:
    def fit(self, X, y=None, **fit_params):
        return self
    def transform(self, X, y=None, **fit_params):
        return X.toarray()
data_pipeline = None
with open('data_pipeline_downloaded', 'rb') as fh:
    data_pipeline = pickle.load(fh)

In [32]:
data_pipeline.transform([['lord ganesa is powerful']])

array([[-0.07029673, -0.03537746, -0.03537746, ..., -0.03537746,
        -0.06820728, -0.05721465]])

In [33]:
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type='ml.m5.large',
                             endpoint_name='xgboost-aws-v1'
                            )

INFO:sagemaker:Creating model with name: xgboost-aws-2024-06-19-09-30-59-058
INFO:sagemaker:Creating endpoint-config with name xgboost-aws-v1
INFO:sagemaker:Creating endpoint with name xgboost-aws-v1


------!

In [34]:
from sagemaker.serializers import CSVSerializer

In [35]:
predictor.serializer = CSVSerializer()

In [36]:
predictor.predict(data_pipeline.transform([['lord ganesa is powerful']]))

b'0.3724002242088318\n'